# Affordability Calculations and Enrichment of Tables

In [5]:
import pandas as pd

rents_PLR = pd.read_csv('csv/rent/rents_PLR.csv')
income_PLR_2023 = pd.read_csv('csv/income/income_PLR_2023.csv')

In [50]:
# Step 1: Filter rents_PLR to only include year 2013
rents_2013 = rents_PLR[rents_PLR['year'] == 2013][['plr_id', 'median', 'cases', 'average']]

# Step 2: Keep only essential columns from income_PLR_2023
income_columns = ['plr_id', 'plr_name', 'bez_id', 'bez_name', 'pgr_id', 'pgr_name',
                  'bzr_id', 'bzr_name', 'svb_vollzeit', 'medianeinkommen']
income_PLR_clean = income_PLR_2023[income_columns].copy()

# Step 3: Merge rent data (left join to keep 542 rows)
merged_df = income_PLR_clean.merge(rents_2013, on='plr_id', how='left')

# Step 4: Rename columns
merged_df = merged_df.rename(columns={
    'medianeinkommen': 'median_gross_income',
    'median': 'median_rent',
    'average': 'average_rent'
})

# Step 5: Create median_net_income (~73% of gross income)
merged_df['median_net_income'] = merged_df['median_gross_income'] * 0.73

# Step 6: Affordability calculations for 1-person household
apartment_size_1R = 50
buergergeld_threshold_m2 = 5.90  # €/m²
buergergeld_monthly = 983        # Monthly Bürgergeld estimate

# Rent burden: % of net income spent on 50m² rent
merged_df['rent_burden_1R'] = (
    merged_df['median_rent'] * apartment_size_1R
) / merged_df['median_net_income']

# Max affordable rent/m² under 30% rule
merged_df['affordable_rent_1R'] = (
    0.30 * merged_df['median_net_income']
) / apartment_size_1R

# Gap between affordable and actual rent
merged_df['gap_rent_1R'] = (
    merged_df['affordable_rent_1R'] - merged_df['median_rent']
)

# Boolean: Is median rent affordable for Bürgergeld recipient?
merged_df['is_affordable_buergergeld_1R'] = (
    merged_df['median_rent'] <= buergergeld_threshold_m2
)

# Boolean: Is rent affordable for average Berliner (net income across PLRs)?
berlin_avg_net_income = merged_df['median_net_income'].mean(skipna=True)
affordable_rent_city_avg = (0.30 * berlin_avg_net_income) / apartment_size_1R

merged_df['is_affordable_city_avg_1R'] = (
    merged_df['median_rent'] <= affordable_rent_city_avg
)


In [55]:
# Converts floats into nullable ints
cols_to_convert = ['bez_id', 'bzr_id', 'pgr_id', 'svb_vollzeit', 'cases']
for col in cols_to_convert:
    if col in merged_df.columns:
        merged_df[col] = merged_df[col].astype('Int64')

In [57]:
merged_df.to_csv('affordability_PLR_2023.csv', index=False)

In [56]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   plr_id                        542 non-null    int64  
 1   plr_name                      542 non-null    object 
 2   bez_id                        540 non-null    Int64  
 3   bez_name                      540 non-null    object 
 4   pgr_id                        540 non-null    Int64  
 5   pgr_name                      540 non-null    object 
 6   bzr_id                        540 non-null    Int64  
 7   bzr_name                      540 non-null    object 
 8   svb_vollzeit                  528 non-null    Int64  
 9   median_gross_income           505 non-null    float64
 10  median_rent                   526 non-null    float64
 11  cases                         525 non-null    Int64  
 12  average_rent                  525 non-null    float64
 13  media

## Adding Population and Calculating the Estimated Available Income to scale the Median Salary.

In [73]:
affordability = pd.read_csv('./csv/affordability_PLR_2023.csv')
combined_pop_2021_2024 = pd.read_csv('./csv/population/combined_pop_2021_2024.csv')
affordability.describe()


,plr_id,bez_id,pgr_id,bzr_id,svb_vollzeit,median_gross_income,median_rent,cases,average_rent,median_net_income,rent_burden_avg_person,rent_burden_buergergeld,population,gap_rent_avg_person,gap_rent_buergergeld
count,5.420000e+02,540.000000,540.000000,540.000000,528.000000,505.000000,526.000000,525.000000,525.000000,505.000000,526.000000,526.000000,539.000000,526.000000,526.000000
mean,6.547167e+06,6.248148,654.777778,65484.281481,1701.032197,4101.366337,7.599620,73.015238,7.725105,2993.997426,0.184996,0.386552,7186.499072,140.280028,-1.701620
std,3.362516e+06,3.380822,336.865044,33686.837766,811.055089,709.391561,1.547537,51.351949,1.473322,517.855840,0.037671,0.078715,3088.260409,1.547537,1.547537
min,1.100101e+06,1.000000,110.000000,11001.000000,97.000000,2806.000000,4.530000,3.000000,4.820000,2048.380000,0.110273,0.230417,15.000000,133.289648,-8.692000
25%,3.601451e+06,3.000000,360.000000,36014.000000,1114.500000,3537.000000,6.500000,34.000000,6.730000,2582.010000,0.158228,0.330621,4804.500000,139.379648,-2.602000
50%,6.300578e+06,6.000000,630.000000,63006.000000,1610.500000,4038.000000,7.375000,63.000000,7.520000,2947.740000,0.179528,0.375127,7215.000000,140.504648,-1.477000
75%,9.301101e+06,9.000000,930.000000,93011.250000,2185.000000,4568.000000,8.500000,105.000000,8.570000,3334.640000,0.206913,0.432350,9428.000000,141.379648,-0.602000
max,1.260124e+07,12.000000,1260.000000,126012.000000,5728.000000,6438.000000,14.590000,347.000000,14.250000,4699.740000,0.355161,0.742116,16689.000000,143.349648,1.368000


In [74]:
# Step 1: Define 2022 official disposable income per capita
official_disposable_income_2022 = 23952 /12  # from Amt für Statistik Berlin-Brandenburg

# Step 2: Apply 5.5% growth to project for 2023
growth_rate = 0.055
projected_disposable_income_2023 = official_disposable_income_2022 * (1 + growth_rate)

# Step 3: Calculate Berlin-wide average gross income (based on your PLR data)
# Only include valid non-null values
berlin_avg_gross_income = affordability['median_gross_income'].mean(skipna=True)

# Step 4: Compute scaling factor to estimate disposable income from gross income
scaling_factor = projected_disposable_income_2023 / berlin_avg_gross_income

print(f"2023 projected available income (per capita): €{projected_disposable_income_2023:.2f}")
print(f"Scaling factor from gross to disposable: {scaling_factor:.3f}")

# Step 5: Create new column with estimated available income per PLR
affordability['estimated_available_income'] = (
    affordability['median_gross_income'] * scaling_factor
)
affordability

2023 projected available income (per capita): €2105.78
Scaling factor from gross to disposable: 0.513


,plr_id,plr_name,bez_id,bez_name,pgr_id,pgr_name,bzr_id,bzr_name,svb_vollzeit,median_gross_income,...,average_rent,median_net_income,is_affordable_avg_person,is_affordable_buergergeld,rent_burden_avg_person,rent_burden_buergergeld,population,gap_rent_avg_person,gap_rent_buergergeld,estimated_available_income
0,1100101,Stülerstraße,1.0,Mitte,110.0,Zentrum,11001.0,Tiergarten Süd,866.0,4721.0,...,9.75,3446.33,True,False,0.219572,0.458800,3680.0,138.859648,-3.122,2423.920851
1,1100102,Großer Tiergarten,1.0,Mitte,110.0,Zentrum,11001.0,Tiergarten Süd,619.0,6279.0,...,12.26,4583.67,True,False,0.321324,0.671414,2040.0,134.679648,-7.302,3223.850672
2,1100103,Lützowstraße,1.0,Mitte,110.0,Zentrum,11001.0,Tiergarten Süd,1250.0,4895.0,...,8.41,3573.35,True,False,0.206913,0.432350,5720.0,139.379648,-2.602,2513.258328
3,1100104,Körnerstraße,1.0,Mitte,110.0,Zentrum,11001.0,Tiergarten Süd,1126.0,5062.0,...,10.24,3695.26,True,False,0.255599,0.534079,4997.0,137.379648,-4.602,2599.001768
4,1100205,Wilhelmstraße,1.0,Mitte,110.0,Zentrum,11002.0,Regierungsviertel,828.0,5384.0,...,13.45,3930.32,True,False,0.340798,0.712106,2960.0,133.879648,-8.102,2764.327443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,12601032,Rollbergesiedlung,12.0,Reinickendorf,1260.0,Märkisches Viertel,126010.0,Rollbergesiedlung,972.0,3148.0,...,5.17,2298.04,True,True,0.123418,0.257884,6115.0,142.809648,0.828,1616.289523
538,12601133,Treuenbrietzener Straße,12.0,Reinickendorf,1260.0,Märkisches Viertel,126011.0,MV Nord,1946.0,2973.0,...,5.83,2170.29,True,True,0.137537,0.287386,11441.0,142.229648,0.248,1526.438612
539,12601134,Märkisches Zentrum,12.0,Reinickendorf,1260.0,Märkisches Viertel,126011.0,MV Nord,2803.0,3151.0,...,5.67,2300.23,True,True,0.139484,0.291455,15947.0,142.149648,0.168,1617.829824
540,12601235,Dannenwalder Weg,12.0,Reinickendorf,1260.0,Märkisches Viertel,126012.0,MV Süd,1774.0,3061.0,...,5.64,2234.53,True,True,0.136806,0.285860,10625.0,142.259648,0.278,1571.620785


In [75]:
#apartment_size_1R = 50

# Rent burden based on available income
#affordability['rent_burden_1R'] = (
#    affordability['median_rent'] * apartment_size_1R
#) / affordability['estimated_available_income']

# Max affordable rent €/m² (30% threshold)
#0affordability['affordable_rent_1R'] = (
#    0.30 * affordability['estimated_available_income']
#) / apartment_size_1R

# Gap between actual and affordable rent
#affordability['gap_rent_1R'] = (
#    affordability['affordable_rent_1R'] - affordability['median_rent']
#)

# Boolean: Is rent affordable under 30% rule?
#affordability['is_affordable_1R'] = (
#    affordability['rent_burden_1R'] <= 0.30
#)

In [76]:
# Step 1: Filter population DataFrame for year 2023
pop_2023 = combined_pop_2021_2024[combined_pop_2021_2024['year'] == 2023]

# Step 2: Select and rename relevant columns
pop_2023 = pop_2023[['plr_id', 'total']].rename(columns={'total': 'population'})

# Step 3: Merge with affordability DataFrame
affordability = affordability.drop(columns=['population'], errors='ignore')
affordability = affordability.merge(pop_2023, on='plr_id', how='left')
affordability

,plr_id,plr_name,bez_id,bez_name,pgr_id,pgr_name,bzr_id,bzr_name,svb_vollzeit,median_gross_income,...,average_rent,median_net_income,is_affordable_avg_person,is_affordable_buergergeld,rent_burden_avg_person,rent_burden_buergergeld,gap_rent_avg_person,gap_rent_buergergeld,estimated_available_income,population
0,1100101,Stülerstraße,1.0,Mitte,110.0,Zentrum,11001.0,Tiergarten Süd,866.0,4721.0,...,9.75,3446.33,True,False,0.219572,0.458800,138.859648,-3.122,2423.920851,3680.0
1,1100102,Großer Tiergarten,1.0,Mitte,110.0,Zentrum,11001.0,Tiergarten Süd,619.0,6279.0,...,12.26,4583.67,True,False,0.321324,0.671414,134.679648,-7.302,3223.850672,2040.0
2,1100103,Lützowstraße,1.0,Mitte,110.0,Zentrum,11001.0,Tiergarten Süd,1250.0,4895.0,...,8.41,3573.35,True,False,0.206913,0.432350,139.379648,-2.602,2513.258328,5720.0
3,1100104,Körnerstraße,1.0,Mitte,110.0,Zentrum,11001.0,Tiergarten Süd,1126.0,5062.0,...,10.24,3695.26,True,False,0.255599,0.534079,137.379648,-4.602,2599.001768,4997.0
4,1100205,Wilhelmstraße,1.0,Mitte,110.0,Zentrum,11002.0,Regierungsviertel,828.0,5384.0,...,13.45,3930.32,True,False,0.340798,0.712106,133.879648,-8.102,2764.327443,2960.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,12601032,Rollbergesiedlung,12.0,Reinickendorf,1260.0,Märkisches Viertel,126010.0,Rollbergesiedlung,972.0,3148.0,...,5.17,2298.04,True,True,0.123418,0.257884,142.809648,0.828,1616.289523,6115.0
538,12601133,Treuenbrietzener Straße,12.0,Reinickendorf,1260.0,Märkisches Viertel,126011.0,MV Nord,1946.0,2973.0,...,5.83,2170.29,True,True,0.137537,0.287386,142.229648,0.248,1526.438612,11441.0
539,12601134,Märkisches Zentrum,12.0,Reinickendorf,1260.0,Märkisches Viertel,126011.0,MV Nord,2803.0,3151.0,...,5.67,2300.23,True,True,0.139484,0.291455,142.149648,0.168,1617.829824,15947.0
540,12601235,Dannenwalder Weg,12.0,Reinickendorf,1260.0,Märkisches Viertel,126012.0,MV Süd,1774.0,3061.0,...,5.64,2234.53,True,True,0.136806,0.285860,142.259648,0.278,1571.620785,10625.0


In [83]:
# Transform columns to integer
cols_to_int = ['population', 'bez_id', 'pgr_id', 'bzr_id', 'svb_vollzeit', 'cases']

for col in cols_to_int:
    if col in affordability.columns:
        affordability[col] = affordability[col].astype('Int64')


# Assumptions
official_disposable_income_2022 = 23952 /12
growth_rate = 0.029
projected_disposable_income_2023 = official_disposable_income_2022 * (1 + growth_rate)
buergergeld_monthly = 983

avg_person_max_rent_m2 = projected_disposable_income_2023 * 0.30 / 50  # ~12.32 €/m²
buergergeld_max_rent_m2 = buergergeld_monthly * 0.30 / 50  # ~5.90 €/m²

affordability['is_affordable_avg_person'] = (affordability['median_rent'] <= avg_person_max_rent_m2)

affordability['is_affordable_buergergeld'] = (affordability['median_rent'] <= buergergeld_max_rent_m2)

# Gap between affordable rent and actual rent for each profile
affordability['gap_rent_avg_person'] = avg_person_max_rent_m2 - affordability['median_rent']
affordability['gap_rent_buergergeld'] = buergergeld_max_rent_m2 - affordability['median_rent']

# Rent burden % for both Average income and Bürgerdelg recipient
affordability['rent_burden_avg_person'] = (affordability['median_rent'] * 50) / projected_disposable_income_2023
affordability['rent_burden_buergergeld'] = (affordability['median_rent'] * 50) / buergergeld_monthly

#affordability = affordability.drop(columns=[
    #'is_affordable_1R',
    #'estimated_available_income',
    #'rent_burden_1R',
    #'affordable_rent_1R',
    #'gap_rent_1R'])

affordability = affordability.rename(columns={'population_x': 'population'})

In [80]:
affordability

,plr_id,plr_name,bez_id,bez_name,pgr_id,pgr_name,bzr_id,bzr_name,svb_vollzeit,median_gross_income,...,cases,average_rent,median_net_income,is_affordable_avg_person,is_affordable_buergergeld,rent_burden_avg_person,rent_burden_buergergeld,gap_rent_avg_person,gap_rent_buergergeld,population
0,1100101,Stülerstraße,1,Mitte,110,Zentrum,11001,Tiergarten Süd,866,4721.0,...,50,9.75,3446.33,True,False,0.018299,0.458800,138.859648,-3.122,3680
1,1100102,Großer Tiergarten,1,Mitte,110,Zentrum,11001,Tiergarten Süd,619,6279.0,...,141,12.26,4583.67,True,False,0.026779,0.671414,134.679648,-7.302,2040
2,1100103,Lützowstraße,1,Mitte,110,Zentrum,11001,Tiergarten Süd,1250,4895.0,...,57,8.41,3573.35,True,False,0.017244,0.432350,139.379648,-2.602,5720
3,1100104,Körnerstraße,1,Mitte,110,Zentrum,11001,Tiergarten Süd,1126,5062.0,...,37,10.24,3695.26,True,False,0.021301,0.534079,137.379648,-4.602,4997
4,1100205,Wilhelmstraße,1,Mitte,110,Zentrum,11002,Regierungsviertel,828,5384.0,...,65,13.45,3930.32,True,False,0.028401,0.712106,133.879648,-8.102,2960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,12601032,Rollbergesiedlung,12,Reinickendorf,1260,Märkisches Viertel,126010,Rollbergesiedlung,972,3148.0,...,180,5.17,2298.04,True,True,0.010285,0.257884,142.809648,0.828,6115
538,12601133,Treuenbrietzener Straße,12,Reinickendorf,1260,Märkisches Viertel,126011,MV Nord,1946,2973.0,...,128,5.83,2170.29,True,True,0.011462,0.287386,142.229648,0.248,11441
539,12601134,Märkisches Zentrum,12,Reinickendorf,1260,Märkisches Viertel,126011,MV Nord,2803,3151.0,...,59,5.67,2300.23,True,True,0.011624,0.291455,142.149648,0.168,15947
540,12601235,Dannenwalder Weg,12,Reinickendorf,1260,Märkisches Viertel,126012,MV Süd,1774,3061.0,...,101,5.64,2234.53,True,True,0.011401,0.285860,142.259648,0.278,10625


In [ ]:
affordability.to_csv('affordability_PLR_2023.csv', index=False)